In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from matplotlib import pyplot 
import seaborn as sns
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test', 'sample_submission', 'sample_submission.csv', 'test.csv', 'train.csv', 'train']


In [97]:
df_train = pd.read_csv('../input/train.csv', parse_dates=['pickup_datetime', 'dropoff_datetime'], index_col='id')

In [3]:
df_train.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
id,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1458644 entries, id2875421 to id1209952
Data columns (total 10 columns):
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null datetime64[ns]
dropoff_datetime      1458644 non-null datetime64[ns]
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: datetime64[ns](2), float64(4), int64(3), object(1)
memory usage: 122.4+ MB


In [5]:
df_train.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


In [100]:
def split_datetime(df):
    df['pickup_year'] = df['pickup_datetime'].dt.year
    df['pickup_month'] = df['pickup_datetime'].dt.month
    df['pickup_day'] = df['pickup_datetime'].dt.day
    df['pickup_hour'] = df['pickup_datetime'].dt.hour
    df['pickup_minute'] = df['pickup_datetime'].dt.minute
    df['pickup_weekday'] = df['pickup_datetime'].dt.weekday
    return df

In [101]:
def data_preporocessing(df):
    df = split_datetime(df)
    df.loc[df['store_and_fwd_flag'] == 'Y' , 'store_and_fwd_flag'] = 1
    df.loc[df['store_and_fwd_flag'] == 'N' , 'store_and_fwd_flag'] = 0
    filtre = (df['trip_duration'] > 120)  &  (df['passenger_count'] > 0) & (df['trip_duration'] < 3600*2)
    df = df[filtre]
    return df
df_train = data_preporocessing(df_train)

In [102]:
df_train.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_weekday
count,1.428060e+06,1.428060e+06,1.428060e+06,1.428060e+06,1.428060e+06,1.428060e+06,1.428060e+06,1.428060e+06,1428060.0,1.428060e+06,1.428060e+06,1.428060e+06,1.428060e+06,1.428060e+06
mean,1.535153e+00,1.666174e+00,-7.397361e+01,4.075087e+01,-7.397354e+01,4.075176e+01,5.475260e-03,8.511625e+02,2016.0,3.517507e+00,1.550564e+01,1.362675e+01,2.959123e+01,3.047668e+00
std,4.987629e-01,1.314854e+00,7.101638e-02,3.261078e-02,7.075255e-02,3.571029e-02,7.379218e-02,6.518150e+02,0.0,1.681006e+00,8.703371e+00,6.390227e+00,1.732147e+01,1.952742e+00
min,1.000000e+00,1.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,0.000000e+00,1.210000e+02,2016.0,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399190e+01,4.073731e+01,-7.399134e+01,4.073578e+01,0.000000e+00,4.110000e+02,2016.0,2.000000e+00,8.000000e+00,9.000000e+00,1.500000e+01,1.000000e+00
50%,2.000000e+00,1.000000e+00,-7.398180e+01,4.075401e+01,-7.397980e+01,4.075447e+01,0.000000e+00,6.740000e+02,2016.0,4.000000e+00,1.500000e+01,1.400000e+01,3.000000e+01,3.000000e+00
75%,2.000000e+00,2.000000e+00,-7.396758e+01,4.076827e+01,-7.396318e+01,4.076969e+01,0.000000e+00,1.084000e+03,2016.0,5.000000e+00,2.300000e+01,1.900000e+01,4.500000e+01,5.000000e+00
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,1.000000e+00,7.191000e+03,2016.0,6.000000e+00,3.100000e+01,2.300000e+01,5.900000e+01,6.000000e+00


In [103]:
TARGET = 'trip_duration'
FEATURES = [
    'passenger_count', 'store_and_fwd_flag','pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
    'pickup_year', 'pickup_month', 'pickup_day', 'pickup_hour', 'pickup_minute', 'pickup_weekday'
]

In [105]:
X_train = df_train[FEATURES]
y_train = df_train[TARGET]

X_train.shape, y_train.shape

((1428060, 12), (1428060,))

In [106]:
rs = ShuffleSplit(n_splits=4, test_size=.15, train_size=.30)

In [107]:
rf = RandomForestRegressor(n_estimators=12, random_state=42)
losses = -cross_val_score(rf, X_train, y_train, cv=rs, scoring='neg_mean_squared_log_error')
losses = [np.sqrt(l) for l in losses]
np.mean(losses)

0.35175437450239333

In [108]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=12, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [110]:
df_test = pd.read_csv('../input/test.csv', index_col='id', parse_dates=['pickup_datetime'])
df_test = split_datetime(df_test)
df_test.loc[df_test['store_and_fwd_flag'] == 'Y' , 'store_and_fwd_flag'] = 1
df_test.loc[df_test['store_and_fwd_flag'] == 'N' , 'store_and_fwd_flag'] = 0

In [112]:
X_test = df_test[FEATURES]

In [113]:
y_pred = rf.predict(X_test)
y_pred.mean()

852.350520448416

In [115]:
submission = pd.read_csv('../input/sample_submission.csv') 
submission['trip_duration'] = y_pred

In [116]:
submission.to_csv('submission.csv', index=False)